In [1]:
library(grid)
library(rpart)
library(rpart.plot)
library(partykit)
library(lattice)
library(ggplot2)
library(caret)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
set.seed(1)

In [3]:
invalids <- c('Time', 'Longitude', 'Latitude', 'Brake', 'Accel', 'flag', 'Curve100', 'Curve150', 'TimeHeadway')
valuables <- c("RoadType","CurveAverage","Speed", "Curve100", "Curve150","MaxSpeed","RiskFactor","Curve","DistSignal","Pitch","AheadDistance","AverageVelocity","TimeToCollision","AccelerationSpeed", "Engine", "SteeringAngle", "TimeHeadway", "Jerk", "LaneCount", "DiffAvgSpeed", "EmptinessOfRoad", "RoadFactor")

In [4]:
printf <- function(...) cat(sprintf(...))

In [5]:
trainAndPredict <- function(dfx, verbose=FALSE) {        
    train <- dfx[, c(valuables, "flag")]
    test <- dfx
    fit <- rpart(flag ~ ., data=train)
    p <- predict(fit, newdata=test, cp=0.018)
    predictedFlags <- colnames(p)[max.col(p, ties.method = "first")]

    showMatrix(test, predictedFlags)

    nerr <- sum(predictedFlags != test$flag)
}

In [6]:
showMatrix <- function(test, predictedFlags) {
    predictedRed <- test[predictedFlags == "Red", ]
    predictedBlue <- test[predictedFlags == "Blue", ]
    print(predictedRed %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
    print(predictedBlue %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
}

In [9]:
createFormula <- function(invalids, allFeatures) {
    isValidColumn <- function(x) ! x %in% invalids
    
    features <- Filter(isValidColumn, allFeatures)
    return(paste("flag ~ ", paste(features, collapse=" + ")))
}

# Predict Red

In [10]:
df3 <- read.csv("../data/middle/sp5.csv", stringsAsFactors=FALSE)

In [11]:
df3$AccelOrBrake[df3$flag == "RedA"] <- "Accel"
df3$AccelOrBrake[df3$flag == "RedB"] <- "Brake"
df3$AccelOrBrake[df3$flag == "BlueA"] <- "Accel"
df3$AccelOrBrake[df3$flag == "BlueB"] <- "Brake"

In [12]:
df3$flag[df3$flag == "RedA"] <- "Red"
df3$flag[df3$flag == "RedB"] <- "Red"
df3$flag[df3$flag == "BlueA"] <- "Blue"
df3$flag[df3$flag == "BlueB"] <- "Blue"
df3$flag <- as.factor(df3$flag)

In [13]:
allFeatures <- c(colnames(df3))
expr <- createFormula(invalids, allFeatures)

In [14]:
df3 %>% group_by(flag, AccelOrBrake) %>% summarize(count=n())

flag,AccelOrBrake,count
Blue,Accel,77
Blue,Brake,372
Red,Accel,401
Red,Brake,192


In [15]:
result <- trainAndPredict(df3, verbose=TRUE)

Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel    43
2   Blue        Brake   142
3    Red        Accel   341
4    Red        Brake   155
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel    34
2   Blue        Brake   230
3    Red        Accel    60
4    Red        Brake    37


In [16]:
fit3 <- rpart(expr, data=df3, method="class")

In [17]:
fit3$cptable[which.min(fit3$cptable[,"xerror"]),"CP"]

[1] 0.01